In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration( gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3500)])

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available()

In [ ]:
# tf.compat.v1.reset_default_graph()

### What does img_augument do?

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()  #50000 train, 10000 test

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
'''
batch size 
'''

batch_size = 50   ## this is what previous tests are done with
# batch_size = 5
autotune=tf.data.experimental.AUTOTUNE
buffer_size = x_train.shape[0]

In [ ]:
category_index=[]     # if want to reduce the training set

for i in range(10):
    index0 = np.where(y_train == i)
    index = index0[0]
    np.random.shuffle(index)
    category_index.append(index)

new_index=[]
for i in range(10):
    new_index.append(category_index[i][0:400])    #################Assign data
new_index=np.concatenate(new_index)
new_index.shape

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=360,width_shift_range=0.3,height_shift_range=0.3,shear_range=0.3)
val_datagen = ImageDataGenerator()

In [ ]:
train_set = train_datagen.flow(x_train[new_index], y_train[new_index], batch_size=batch_size)
val_set = val_datagen.flow(x_test, y_test, batch_size=200)

### model

In [ ]:
from models import ANN_model

model=ANN_model()

### train model

In [ ]:
# es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500)

In [ ]:
# lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.8, min_lr=0.0001)

In [ ]:
history=model.fit(train_set,
                  epochs=200,
                  steps_per_epoch=new_index.shape[0]//batch_size,
                  validation_data=val_set,
                  validation_steps=10000//200,
#                   callbacks=[es_callback],
                  verbose=1)
#                   callbacks=[es_callback,lr_callback])

### view history

In [ ]:
plt.plot(history.epoch, history.history.get('val_acc'),label='val_acc')
plt.plot(history.epoch, history.history.get('acc'), label='acc')
plt.legend()

In [ ]:
plt.plot(history.epoch, history.history.get('val_loss'), label='val_loss')
plt.plot(history.epoch, history.history.get('loss'), label='loss')
plt.legend()

In [ ]:
# model.evaluate(val_set,steps=10000//batch_size)

In [ ]:
np.array(model.get_weights()).shape

In [ ]:
epo_h=history.epoch
loss_h=history.history.get('val_loss')
acc_h=history.history.get('val_acc')

arr = np.array([epo_h,loss_h,acc_h])

In [ ]:
# filename=os.path.join('result/zCentral','new_{}_central.batch{}.npy'.format(new_index.shape[0],batch_size))
# np.save(filename,arr)
# print(filename)